# Jupyter Notebook para implenetação teste de Transfer Learning
### Algoritmo utilizando apenas 2% do conjutno de dados

#### Importação de bibliotecas

In [1]:
from keras import applications
from keras.optimizers import SGD
from keras.models import Model
from keras.layers import Dense, Flatten
import numpy as np
from sklearn.model_selection import train_test_split
from keras import backend as K

C:\Users\giovanadelucca\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


#### Declarando variáveis de parâmetros

In [4]:
input_channels = 3 #quantidade de canais de cores na camada de entrada
output_channels = 2 #quantidade de canais de cores na camada de saída

test_size = 0.3 #porcentagem do dataset para utilizar nos testes
dim = 128 #dimensão das imagens utilizadas como exemplos
flatten = dim*dim*output_channels #dimensão da camada de saída
momentum = 0.9 #valor do momentum
epochs = 3 #quantidade de épocas
lr = 0.0001 #taxa de aprendizado

#observação: em caso de alterações, não esquecer de alterar os parâmetros na string da próxima célula

#### Salvando parâmetros utilizados em string

In [5]:
params="""
input_channels = 3 #quantidade de canais de cores na camada de entrada
output_channels = 2 #quantidade de canais de cores na camada de saída

test_size = 0.3 #porcentagem do dataset para utilizar nos testes
dim = 128 #dimensão das imagens utilizadas como exemplos
flatten = dim*dim*output_channels #dimensão da camada de saída
momentum = 0.9 #valor do momentum
epochs = 3 #quantidade de épocas
lr = 0.0001 #taxa de aprendizado
"""

#### Leitura do conjunto de dados

In [6]:
X = np.load('x.npy')
Y = np.load('y.npy')

#### Separação do conjunto de dados para treinamento e testes

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=test_size)

#### Modelo de Transfer Learning 

In [8]:
model = applications.vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(dim,dim,input_channels), pooling=None)

#### Ajustes do modelo

In [9]:
#adding custom layers 
x = model.output
x = Dense(1024, activation='relu')(x)
x = Flatten()(x)
predictions = Dense(flatten, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = model.input, outputs = predictions)

#freezing layers
for layer in model.layers:
    layer.trainable = False
    
# compile the model 
model_final.compile(loss = 'mean_squared_error', optimizer=SGD(lr=lr, momentum=momentum))

#### Treinamento do modelo

In [14]:
model_final.fit_generator((x_train,y_train),steps_per_epoch=len(X), epochs=epochs, verbose=0)

NameError: name 'X' is not defined

#### Obtendo as métricas de desempenho calculadas

In [18]:
loss = model_final.evaluate(x_test, y_test)
print('loss',loss)
out_loss = '\n\nLoss: '+str(loss)

InvalidArgumentError: Node 'IsVariableInitialized_94': Unknown input node 'dense_1/kernel'

#### Salvando parâmetros utilizados e resultados obtidos  em um arquivo

In [12]:
out = params+out_loss

arq = open('output.txt', 'w')
arq.write(out)
arq.close()